In [94]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRe

In [95]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-14 15:14:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.07MB/s    in 0.2s    

2023-02-14 15:14:34 (4.07 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [96]:
# Start a SparkSession
import findspark
findspark.init()

In [97]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MobileCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Mobile Reviews

In [107]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://amazonreviews011.s3.eu-west-2.amazonaws.com/amazon_reviews_us_Mobile_Electronics_v1_00.tsv" 
spark.sparkContext.addFile(url)
Mobile_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv"), sep="\t", header=True)
# Show DataFrame
Mobile_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [108]:
Mobile_df.count()

104975

In [109]:
df_mobile = Mobile_df.dropna()
df_mobile.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [110]:
# Final count of records in df_camera
df_mobile.count()

104972

In [111]:
df_mobile.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [112]:
# Create dataframe to match review_id table in PostgreSQL/RDS
review_id_df_mobile = df_mobile.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df_mobile.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8MEA6IGAHO0B|   20422322|B00MC4CED8|     217304173| 2015-08-31|
|R31LOQ8JGLPRLK|   40835037|B00OQMFG1Q|     137313254| 2015-08-31|
|R2Y0MM9YE6OP3P|   51469641|B00QERR5CY|      82850235| 2015-08-31|
| RRB9C05HDOD4O|    4332923|B00QUFTPV4|     221169481| 2015-08-31|
|R26I2RI1GFV8QG|   44855305|B0067XVNTG|     563475445| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [113]:
review_id_df_mobile = review_id_df_mobile.dropDuplicates(["review_id"])

In [114]:
# Convert dataframe dtype to match review_id table schema dtypes
from pyspark.sql.types import * 

review_id_df_mobile = review_id_df_mobile.withColumn("customer_id",review_id_df_mobile["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_df_mobile["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_df_mobile["review_date"].cast(DateType()))

review_id_df_mobile.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [115]:
# Create dataframe to match products table in PostgreSQL/RDS
products_df_mobile = df_mobile.select(["product_id", "product_title"])
products_df_mobile.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00MC4CED8|BlackVue DR600GW-PMP|
|B00OQMFG1Q|GENSSI GSM / GPS ...|
|B00QERR5CY|iXCC Multi pack L...|
|B00QUFTPV4|abcGoodefg® FBI C...|
|B0067XVNTG|Generic Car Dashb...|
+----------+--------------------+
only showing top 5 rows



In [116]:
products_df_mobile = products_df_mobile.dropDuplicates(["product_id"])

In [117]:
products_df_mobile.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [118]:
# Create dataframe to match customers table in PostgreSQL/RDS
customers_df_mobile = df_mobile.select(["customer_id"])
customers_df_mobile.show(5)

+-----------+
|customer_id|
+-----------+
|   20422322|
|   40835037|
|   51469641|
|    4332923|
|   44855305|
+-----------+
only showing top 5 rows



In [119]:
customers_df_mobile = customers_df_mobile.groupBy("customer_id").count()
customers_df_mobile.orderBy("customer_id").select(["customer_id", "count"])
customers_df_mobile.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    3962209|    1|
|   24644096|    1|
|   24659376|    1|
|   24486730|    2|
|   36657077|    1|
+-----------+-----+
only showing top 5 rows



In [121]:
customers_df_mobile = customers_df_mobile.withColumnRenamed("count", "customer_count")
customers_df_mobile.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    3962209|             1|
|   24644096|             1|
|   24659376|             1|
|   24486730|             2|
|   36657077|             1|
+-----------+--------------+
only showing top 5 rows



In [122]:
customers_df_mobile = customers_df_mobile.withColumn("customer_id",customers_df_mobile["customer_id"].cast(IntegerType()))\
    .withColumn("customer_count",customers_df_mobile["customer_count"].cast(IntegerType()))

customers_df_mobile.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [124]:
# Create dataframe to match vine table in PostgreSQL/RDS
vine_df_mobile = df_mobile.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df_mobile.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8MEA6IGAHO0B|          5|            0|          0|   N|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|
| RRB9C05HDOD4O|          4|            0|          0|   N|
|R26I2RI1GFV8QG|          2|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [125]:
vine_df_mobile = vine_df_mobile.dropDuplicates(["review_id"])

In [126]:
vine_df_mobile = vine_df_mobile.withColumn("star_rating",vine_df_mobile["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine_df_mobile["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine_df_mobile["total_votes"].cast(IntegerType()))

vine_df_mobile.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



Postgres Setup and writing to database

In [128]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cmmggpkmm8qx.eu-west-2.rds.amazonaws.com:5432/AmazonReview_Mobile"
config = {"user":"root", 
          "password": "password1", 
          "driver":"org.postgresql.Driver"}

In [129]:
review_id_df_mobile.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [130]:
products_df_mobile.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [131]:
customers_df_mobile.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [132]:
vine_df_mobile.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)